#Load drive to colab

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#import libraries

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD


#Loading the datasets

In [5]:
users = pd.read_csv('/content/drive/MyDrive/dataset1/book rental datasets (1)/BX-Users.csv', sep=',', error_bad_lines=False, encoding='latin-1')
books = pd.read_csv('/content/drive/MyDrive/dataset1/book rental datasets (1)/BX-Books.csv', sep=',', error_bad_lines=False, encoding='latin-1')
ratings = pd.read_csv('/content/drive/MyDrive/dataset1/book rental datasets (1)/BX-Book-Ratings.csv', sep=',', error_bad_lines=False, encoding='latin-1', nrows=10000)

<ipython-input-5-fb7b58fa2d1a>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  users = pd.read_csv('/content/drive/MyDrive/dataset1/book rental datasets (1)/BX-Users.csv', sep=',', error_bad_lines=False, encoding='latin-1')
<ipython-input-5-fb7b58fa2d1a>:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  users = pd.read_csv('/content/drive/MyDrive/dataset1/book rental datasets (1)/BX-Users.csv', sep=',', error_bad_lines=False, encoding='latin-1')
<ipython-input-5-fb7b58fa2d1a>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  books = pd.read_csv('/content/drive/MyDrive/dataset1/book rental datasets (1)/BX-Books.csv', sep=',', error_bad_lines=False, encoding='latin-1')
<ipython-input-5-fb7b58fa2d1a>:2: DtypeWarning: Columns (3) have mixed types. S

#understanding the Dataset

In [6]:
users.head()

,user_id,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [7]:
users.dtypes

user_id      object
Location     object
Age         float64
dtype: object

In [8]:
books.head()

,isbn,book_title,book_author,year_of_publication,publisher
0,195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,2005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,60973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


In [9]:
books.dtypes

isbn                   object
book_title             object
book_author            object
year_of_publication    object
publisher              object
dtype: object

In [10]:
ratings.head()

,user_id,isbn,rating
0,276725,034545104X,0
1,276726,155061224,5
2,276727,446520802,0
3,276729,052165615X,3
4,276729,521795028,6


In [11]:
ratings.dtypes

user_id     int64
isbn       object
rating      int64
dtype: object

In [12]:
print(users.shape)
print(books.shape)
print(ratings.shape)

(278859, 3)
(271379, 5)
(10000, 3)


#starting data processing

In [13]:
print("NaN values ​​in Users:", users.isnull().sum().sum())
print("NaN values ​​in Books:", books.isnull().sum().sum())
print("NaN values ​​in Ratings:", ratings.isnull().sum().sum())

NaN values ​​in Users: 110764
NaN values ​​in Books: 3
NaN values ​​in Ratings: 0


lets start with Users dataset

# create a mask because we have errors with user_id

In [14]:
# Create a boolean mask for rows with non-integer values in user_id
mask = pd.to_numeric(users['user_id'], errors='coerce').isna()
df_not_int = users[mask]
print(df_not_int)

                user_id Location  Age
275081  , milan, italy"      NaN  NaN


drop the error row

In [15]:
users = users.drop(275081)

fill 'Age' null values with 0

In [16]:
# Replacing NaN with '0' in specific column
users['Age'] = users['Age'].fillna(0)
print("NaN values ​​in Users:", users.isnull().sum().sum())

NaN values ​​in Users: 0


back 'user_id' for int

In [21]:
#First let's adjust the type of the columns
# Converting 'user_id' to int
users['user_id'] = users['user_id'].astype(int)

# Converting 'Age' to int
users['Age'] = users['Age'].astype(int)

In [22]:

print("NaN values ​​in Users:", users.isnull().sum().sum())

NaN values ​​in Users: 0


In [23]:
# Checking the exact location of null values in the dataset
null_values = users[users.isnull().any(axis=1)]
# Displaying rows with null values
print("Rows with null values:")
print(null_values)

Rows with null values:
Empty DataFrame
Columns: [user_id, Location, Age]
Index: []


moving back the error row with adjusted values

In [24]:
# Finding the next available 'user_id'
new_user_id = users['user_id'].max() + 1

# Adding the new user automatically (replace with real data)
new_user = {'user_id': new_user_id, 'Location': 'Milan, Italia', 'Age': 0}
users = users.append(new_user, ignore_index=True)

# Checking the result
print(users.tail())

        user_id                   Location  Age
278855   278856  brampton, ontario, canada    0
278856   278857  knoxville, tennessee, usa    0
278857   278858       dublin, n/a, ireland    0
278858   278859              Milan, Italia    0
278859   278860              Milan, Italia    0


<ipython-input-24-5d0b989f3744>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  users = users.append(new_user, ignore_index=True)


In [25]:
print(users.shape)

(278860, 3)


#Now lets cleanning 'books' data set

In [26]:
# Checking the exact location of null values in the dataset
null_values2 = books[books.isnull().any(axis=1)]
# Displaying rows with null values
print("Rows with null values:")
print(null_values2)

Rows with null values:
              isbn                                         book_title  \
128896  193169656X                                        Tyrant Moon   
129043  1931696993                                    Finders Keepers   
187700  9627982032  The Credit Suisse Guide to Managing Your Perso...   

            book_author year_of_publication                       publisher  
128896  Elaine Corvidae                2002                             NaN  
129043  Linnea Sinclair                2001                             NaN  
187700              NaN                1995  Edinburgh Financial Publishing  


let's fill in with no information as it does not affect the study of the dataset


In [27]:
books = books.fillna('NO SOURCE')

In [28]:
print("NaN values ​​in Books:", books.isnull().sum().sum())

NaN values ​​in Books: 0


#'#############################################'

# cleaning data finished

#'#############################################'

•	Read the data where ratings are given by users

In [29]:
# first few rows of the ratings DataFrame
print(ratings.head())

   user_id        isbn  rating
0   276725  034545104X       0
1   276726   155061224       5
2   276727   446520802       0
3   276729  052165615X       3
4   276729   521795028       6


lets merging the two datasets books and rating

In [30]:
df = pd.merge(ratings, books, on = 'isbn')
df

,user_id,isbn,rating,book_title,book_author,year_of_publication,publisher
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
1,276726,155061224,5,Rites of Passage,Judith Rae,2001,Heinle
2,276727,446520802,0,The Notebook,Nicholas Sparks,1996,Warner Books
3,278418,446520802,0,The Notebook,Nicholas Sparks,1996,Warner Books
4,276729,052165615X,3,Help!: Level 1,Philip Prowse,1999,Cambridge University Press
...,...,...,...,...,...,...,...
8696,243,385720106,7,A Map of the World,Jane Hamilton,1999,Anchor Books/Doubleday
8697,243,425092917,0,The Accidental Tourist,Anne Tyler,1994,Berkley Publishing Group
8698,243,425098834,0,If Morning Ever Comes,Anne Tyler,1983,Berkley Publishing Group
8699,243,425163407,9,Unnatural Exposure,Patricia Daniels Cornwell,1998,Berkley Publishing Group


•	Take a quick look at the number of unique users and books

In [31]:
# Number of unique users
unique_users = ratings['user_id'].nunique()

# Number of unique books
unique_books = ratings['isbn'].nunique()

# Display the results
print(f"Number of unique users: {unique_users}")
print(f"Number of unique books: {unique_books}")


Number of unique users: 941
Number of unique books: 9335


•	Convert ISBN variables to numeric numbers in the correct order

In [85]:
df['isbn_numeric'] = pd.factorize(df['isbn'])[0]


In [86]:
df['user_id_numeric'] = pd.factorize(df['user_id'])[0]

In [87]:
df

,user_id_numeric,isbn_numeric,rating,book_title,book_author,year_of_publication,publisher,user_id,isbn
0,0,0,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,276725,034545104X
1,1,1,5,Rites of Passage,Judith Rae,2001,Heinle,276726,155061224
2,2,2,0,The Notebook,Nicholas Sparks,1996,Warner Books,276727,446520802
3,3,2,0,The Notebook,Nicholas Sparks,1996,Warner Books,278418,446520802
4,4,3,3,Help!: Level 1,Philip Prowse,1999,Cambridge University Press,276729,052165615X
...,...,...,...,...,...,...,...,...,...
8696,96,8046,7,A Map of the World,Jane Hamilton,1999,Anchor Books/Doubleday,243,385720106
8697,96,8047,0,The Accidental Tourist,Anne Tyler,1994,Berkley Publishing Group,243,425092917
8698,96,8048,0,If Morning Ever Comes,Anne Tyler,1983,Berkley Publishing Group,243,425098834
8699,96,8049,9,Unnatural Exposure,Patricia Daniels Cornwell,1998,Berkley Publishing Group,243,425163407


 Split your data into two sets (training and testing).

In [93]:

train, test = train_test_split(df, test_size=.30, random_state = 10)

In [94]:
train.shape

(6090, 9)

In [95]:
test.shape

(2611, 9)

In [96]:
test.head()

,user_id_numeric,isbn_numeric,rating,book_title,book_author,year_of_publication,publisher,user_id,isbn
7794,3,7160,0,Aerie Tik Tok of Oz: Defiant-Cn16dp,Baum,1988,Tor Books,278418,1559029897
5451,3,4839,0,"Wild Honey (Harlequin Super Romance, No 731)",Veronica Sattler,1997,Harlequin,278418,373707312
8288,53,7640,0,Click,Dan Whipple,2001,University Press of Colorado,278633,870816527
2853,465,2373,0,The Devil's Arithmetic,Jane Yolen,1990,Puffin Books,277814,140345353
5348,3,4736,0,Way We Wed (A Year Of Loving Dangerously) (Sil...,Pat Warren,2001,Silhouette,278418,373271409


make the predictions

In [97]:
# Getting the total number of users and books
n_users = max(max(train['user_id_numeric']), max(test['user_id_numeric']))
n_books = max(max(train['isbn_numeric']), max(test['isbn_numeric']))

In [98]:
# Initializing matrices with zeros
train_matrix = np.zeros((n_users, n_books))
test_matrix = np.zeros((n_users, n_books))

In [99]:
# Filling the matrices with evaluations from the training set
for row in train.itertuples():
    train_matrix[row[1] - 1, row[2] - 1] = row[3]

In [100]:
# Filling the matrices with evaluations from the test set
for row in test.itertuples():
    test_matrix[row[1] - 1, row[2] - 1] = row[3]

In [104]:
from sklearn.metrics.pairwise import pairwise_distances
user_correlation = 1 - pairwise_distances(train_matrix, metric='cosine')
item_correlation = 1 - pairwise_distances(train_matrix.T, metric='cosine')

In [105]:
user_correlation

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [106]:
item_correlation

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [110]:
# Function to make predictions
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        # We use np.newaxis to ensure that mean_user_rating has the same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

In [116]:
# Make predictions using user similarity
user_prediction = predict(train_matrix, user_correlation, type='user')
item_prediction = predict(train_matrix, item_correlation, type = 'item')

In [117]:
item_prediction.shape

(827, 8050)

In [128]:
user_prediction.shape

(827, 8050)

In [155]:
# Choose a specific user and item to predict
user_id = 0
item_id = 0

# Choose a specific user and item to predict
item_ratings = train_matrix[:, item_id]

# Calculate the similarity of the chosen user with all other users
user_similarity = user_correlation[user_id, :]

# Weight other users' reviews by similarity
weighted_ratings = user_similarity * item_ratings

# Normalize the weights to get a valuation prediction
prediction = np.sum(weighted_ratings) / np.sum(np.abs(user_similarity))

print(f"Rating Forecast for the Item {item_id} by the User {user_id}: {prediction}")

Rating Forecast for the Item 0 by the User 0: 5.0


In [156]:
print(prediction)

5.0


In [165]:
# fuction for calculate RMSE
def calculate_rmse(predictions, actual):
    # Find the non-zero positions in real reviews
    non_zero_positions = actual.nonzero()

    # Get predictions only for non-zero positions
    predicted_values = predictions[non_zero_positions]

    # Get real values ​​only for non-zero positions
    actual_values = actual[non_zero_positions]

    # Calculate an RMSE
    rmse = np.sqrt(mean_squared_error(actual_values, predicted_values))

    return rmse

In [167]:
from sklearn.metrics import mean_squared_error

In [168]:
# predictions in 'user_prediction', 'item_prediction' and the test set in 'test_matrix
rmse_user = calculate_rmse(user_prediction, test_matrix)
rmse_item = calculate_rmse(item_prediction, test_matrix)


In [170]:
print(f"RMSE using User Similarity: {rmse_user}")
print(f"RMSE using User Similarity: {rmse_item}")

RMSE using User Similarity: 7.629591994377947
RMSE using User Similarity: 7.632442731110491
